<a href="https://colab.research.google.com/github/ricoen/google-ee-data-extraction/blob/main/ee_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine Data Extract

Author [@ricoen](https://github.com/ricoen)<br>

MIT Licensed

In [ ]:
#Import Earth Engine
import ee

In [ ]:
#Authenticate EE with your account
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=inCUuZaCpDotIIOVmnMqm7F0AM48Esw3p8-vpMfGIic&tc=WT2mXtEQIQ93SXLSKSaDgsxECqf9UEF0eXU8QBMADXo&cc=o6oG5GAggCcLiRc4L5fQJJYI03TvYuKGZPy4LAWGshM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjXeeXz-v3VMp__Hz5jUQvDn0ykHuGoInXlyJ1A80MD-PAtLfbU7wc

Successfully saved authorization token.


In [ ]:
#Initialize EE
ee.Initialize()

In [ ]:
#Set the dataset from EE Image Collection
dataset = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")

In [ ]:
#Set the dataset taken time
taken = dataset.filterDate("2022-07-01", "2022-07-31")

In [ ]:
#Get the first raster for visualize the data
first_raster = ee.Image(taken.first())

In [ ]:
#Get info from the selected raster
img = first_raster

img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'Albedo_inst',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'AvgSurfT_inst',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'CanopInt_inst',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'ECanop_tavg',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'ESoil_tavg',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'Evap_tavg'

In [ ]:
#Get the temperature value from T band
temperature = img.select("Tair_f_inst")

In [ ]:
#Import the folium library.
import folium
from folium import plugins

In [ ]:
#Set the basemap for map
basemaps = {
    "Google Maps": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Maps",
        overlay = True,
        control = True
    ),
    "Google Satellite": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Satellite",
        overlay = True,
        control = True
    )
}

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  
  folium.raster_layers.TileLayer(
    tiles = map_id_dict["tile_fetcher"].url_format,
    attr = "Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-7.50, 111.25], zoom_start=3, height=1000)

# Add custom basemaps
basemaps["Google Satellite"].add_to(my_map)

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Temperature Params
temp_params = {
  "min": 206,
  "max": 328,
  "palette": ["green", "yellow", "red"]}

# Add the elevation model to the map object.
my_map.add_ee_layer(temperature.updateMask(temperature.gt(0)), temp_params, "Air Temperature")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
#Set the AOI (Area of Interest)
aoi = ee.Geometry.Polygon([
          [
            [
              112.62535572052002,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.988390117845741
            ]
          ]
        ], None, False)

#poi = ee.Geometry.Point([112.644105, -7.9514187])

In [ ]:
#Reduce the raster
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), aoi)
    return image.set(dict)

In [ ]:
#Calculate the raster property
withMean = taken.map(setProperty)

In [ ]:
import pandas as pd

#Convert temperature unit (Kelvin) to Celcius 
t_avg = withMean.aggregate_array("AvgSurfT_inst").getInfo()
t_c = [x - 273 for x in t_avg]
t_c = pd.Series(t_c).round(decimals=0)

#print(t_c)

In [ ]:
#Get the date
from datetime import datetime as dt

time = withMean.aggregate_array("system:time_start").getInfo()
#Convert from miliseconds to true time (local time with 24-H format)
date = [x // 1000 for x in time]
captured = [dt.utcfromtimestamp(i).strftime("%c GMT") for i in date]
captured = pd.Series(captured)
converted_date = pd.to_datetime(captured).dt.tz_convert("Asia/Jakarta")

# print(captured)

In [ ]:
#Just look inside the array
t_c

0      21.0
1      30.0
2      30.0
3      23.0
4      17.0
       ... 
231    28.0
232    33.0
233    25.0
234    18.0
235    15.0
Length: 236, dtype: float64

In [ ]:
#Just look inside the array
converted_date

0     2022-07-01 07:00:00+07:00
1     2022-07-01 10:00:00+07:00
2     2022-07-01 13:00:00+07:00
3     2022-07-01 16:00:00+07:00
4     2022-07-01 19:00:00+07:00
                 ...           
231   2022-07-30 10:00:00+07:00
232   2022-07-30 13:00:00+07:00
233   2022-07-30 16:00:00+07:00
234   2022-07-30 19:00:00+07:00
235   2022-07-31 04:00:00+07:00
Length: 236, dtype: datetime64[ns, Asia/Jakarta]

In [ ]:
#Visualize The Data

#Convert from numpy array to Pandas Series
date = pd.Series(converted_date)
day = date.dt.day_name()
temperature = pd.Series(t_c)
#Concatnate the columns of Date and Temperature
df = pd.concat([day, date, temperature], axis=1)
df.columns = ["day", "date", "temperature"]

df

,day,date,temperature
0,Friday,2022-07-01 07:00:00+07:00,21.0
1,Friday,2022-07-01 10:00:00+07:00,30.0
2,Friday,2022-07-01 13:00:00+07:00,30.0
3,Friday,2022-07-01 16:00:00+07:00,23.0
4,Friday,2022-07-01 19:00:00+07:00,17.0
...,...,...,...
231,Saturday,2022-07-30 10:00:00+07:00,28.0
232,Saturday,2022-07-30 13:00:00+07:00,33.0
233,Saturday,2022-07-30 16:00:00+07:00,25.0
234,Saturday,2022-07-30 19:00:00+07:00,18.0


In [ ]:
#Visualize the data with Plotly
import plotly.express as px

#Define x and y values
fig = px.line(df, x="date", y="temperature", 
              title="Surface Temperature on Malang City - GLDAS-2.1 (NASA)")
#Show the figure
fig.show()